In [1]:
import pandas as pd
from tabulate import tabulate

# Function to load items from a CSV file
def load_items_from_csv(filename):
    try:
        items_df = pd.read_csv(filename)
        return items_df
    except FileNotFoundError:
        print("Item file not found.")
        return pd.DataFrame()

# Function to save items to a CSV file
def save_items_to_csv(items_df, filename):
    items_df.to_csv(filename, index=False)
    print("Items saved to", filename)

# Function to display inventory
def display_inventory(items_df):
    print("\nInventory:")
    print(items_df)

# Function to display inventory in a formatted table using tabulate
def display_inventory_table(items_df):
    print("\nInventory:")
    print(tabulate(items_df, headers='keys', tablefmt='fancy_grid'))

# Function to add items to the inventory
def add_items(items_df):
    try:
        item_no = input("Enter the item number: ")
        if not items_df.empty and item_no in items_df['Item No'].astype(str).values:
            print("Item number already exists. Please choose another item number.")
            return items_df
        item_name = input("Enter the item name: ")
        category = input("Enter the category: ")
        quantity = int(input("Enter the quantity: "))
        price = float(input("Enter the price: "))
        
        new_item = pd.DataFrame([[item_no, item_name, category, quantity, price]], 
                                columns=['Item No', 'Item Name', 'Category', 'Quantity', 'Price'])
        
        items_df = pd.concat([items_df, new_item], ignore_index=True)
        print("Item added to inventory.")
        return items_df
    except ValueError:
        print("Invalid input. Please enter valid values.")

# Function to add this to the given code and give the complete code
def customer_view_inventory(items_df):
    display_inventory_table(items_df)

# Function to add items to cart
def add_to_cart(items_df, cart):
    while True:
        item_no = input("Enter the item number to add to cart (enter '0' to stop adding items): ")
        if item_no == '0':
            break
        item = items_df.loc[items_df['Item No'].astype(str) == str(item_no)]
        if not item.empty:
            available_quantity = item.iloc[0]['Quantity']
            item_name = item.iloc[0]['Item Name']
            quantity = int(input("Enter the quantity: "))
            if quantity > 0 and quantity <= available_quantity:
                price = item.iloc[0]['Price']
                cart.append({"Item No": item_no, "Item Name": item_name, "Category": item.iloc[0]['Category'], "Quantity": quantity, "Price": price})
                print(f"{quantity} {item_name}(s) added to the cart.")
                items_df.loc[items_df['Item No'].astype(str) == str(item_no), 'Quantity'] -= quantity
            elif quantity > available_quantity:
                print(f"Sorry, only {available_quantity} {item_name} available.")
            else:
                print("Quantity should be greater than zero.")
        else:
            print("Invalid item number or item not found in inventory.")

# Function to generate bill
def generate_bill(cart):
    # Import tabulate here to fix the issue
    from tabulate import tabulate

    cart_df = pd.DataFrame(cart)
    cart_df['Subtotal'] = cart_df['Quantity'] * cart_df['Price']
    total = cart_df['Subtotal'].sum()
    gst = total * 0.18  # Assuming GST rate is 18%
    total_with_gst = total + gst

    # Create a new DataFrame to include total
    total_row = pd.DataFrame({"Item No": [''], "Item Name": [''], "Category": [''], "Quantity": ['Total:'], "Price": ['']}, index=[len(cart_df)])
    total_row['Subtotal'] = total
    cart_df = pd.concat([cart_df, total_row])

    print("\nBEMS Supermarket - Invoice\n")
    print(tabulate(cart_df[:-1], headers='keys', tablefmt='fancy_grid', showindex=False, numalign="right"))
    print("\nTotal (excluding GST):", total, "rupees")
    print("GST (18%):", gst, "rupees")
    print("\033[1;30;47mTotal (including GST):", total_with_gst, "rupees\033[m")  # Bold color box for total

    # Save the cart to a CSV file
    save_items_to_csv(cart_df[:-1], "customer_cart.csv")  # Excluding the total row

# Load items from CSV file
items_filename = "inventory.csv"
items_df = load_items_from_csv(items_filename)

# Create an empty list to store the items that the customer buys
cart = []

while True:
    # Welcome message and options for the user
    print("\nWelcome to BEMS Supermarket!")
    print("Please select an option:")
    print("1. User")
    print("2. Customer")
    print("3. Exit")

    # Asking for user's choice
    user_choice = input("Enter your choice (1, 2, or 3): ")

    if user_choice == '1':  # User
        while True:
            print("\nUser Options:")
            print("1. View Inventory")
            print("2. Add Items")
            print("3. Upload to CSV")
            print("4. Exit")
            user_option = input("Enter your choice (1, 2, 3, or 4): ")

            if user_option == '1':
                display_inventory_table(items_df)
            elif user_option == '2':
                items_df = add_items(items_df)
            elif user_option == '3':
                save_items_to_csv(items_df, items_filename)
            elif user_option == '4':
                break
            else:
                print("Invalid choice. Please enter '1', '2', '3', or '4'.")

    elif user_choice == '2':  # Customer
        while True:
            print("\nCustomer Options:")
            print("1. View Inventory")
            print("2. Add to Cart")
            print("3. Generate Bill")
            print("4. Exit")

            customer_option = input("Enter your choice (1, 2, 3, or 4): ")

            if customer_option == '1':
                customer_view_inventory(items_df)
            elif customer_option == '2':
                add_to_cart(items_df, cart)
            elif customer_option == '3':
                generate_bill(cart)
                print("Thank you for shopping!")
                break
            elif customer_option == '4':
                break
            else:
                print("Invalid choice. Please enter '1', '2', '3', or '4'.")

    elif user_choice == '3':  # Exit
        print("Exiting BEMS Supermarket.")
        break

    else:
        print("Invalid choice. Please enter '1', '2', or '3'.")



Welcome to BEMS Supermarket!
Please select an option:
1. User
2. Customer
3. Exit
Enter your choice (1, 2, or 3): 1

User Options:
1. View Inventory
2. Add Items
3. Upload to CSV
4. Exit
Enter your choice (1, 2, 3, or 4): 1

Inventory:
╒════╤═══════════╤═════════════╤════════════╤════════════╤═════════╕
│    │   Item No │ Item Name   │ Category   │   Quantity │   Price │
╞════╪═══════════╪═════════════╪════════════╪════════════╪═════════╡
│  0 │         1 │ milk        │ dairy      │         20 │      25 │
├────┼───────────┼─────────────┼────────────┼────────────┼─────────┤
│  1 │         2 │ cake        │ bakery     │         30 │     100 │
├────┼───────────┼─────────────┼────────────┼────────────┼─────────┤
│  2 │         3 │ butter      │ dairy      │         23 │      23 │
├────┼───────────┼─────────────┼────────────┼────────────┼─────────┤
│  3 │         4 │ lipsticks   │ cosmetics  │         50 │      45 │
├────┼───────────┼─────────────┼────────────┼────────────┼─────────┤
│  4